In [1]:
%matplotlib notebook
import os
import pandas as pd
import numpy as np
import pymongo

In [2]:
file_to_load = "../../Resources/casos-asociados-a-covid-19.csv"

covid_cases_data = pd.read_csv(file_to_load)
covid_cases_data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (30,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FECHA ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD UNIDAD MEDICA,SEXO,ENTIDAD NACIMIENTO,ENTIDAD RESIDENCIA,MUNICIPIO RESIDENCIA,TIPO PACIENTE,...,Negativo,MIGRANTE,PAIS NACIONALIDAD,PAIS ORIGEN,UNIDAD DE CUIDADOS INTENSIVOS,RANGO EDAD,num_fallecidos,num_hospitalizados,positivo,pendiente
0,2020-06-24,076847,USMER,SSA,PUEBLA,MUJER,PUEBLA,PUEBLA,NaN,AMBULATORIO,...,NaN,NO ESPECIFICADO,México,99,NO APLICA,31-40,NaN,NaN,Positivo SARS-CoV-2,NaN
1,2020-06-24,0f2b37,USMER,SSA,CHIAPAS,HOMBRE,CHIAPAS,CHIAPAS,NaN,AMBULATORIO,...,NaN,NO ESPECIFICADO,México,99,NO APLICA,31-40,NaN,NaN,Positivo SARS-CoV-2,NaN
2,2020-06-24,1aa59e,USMER,SSA,VERACRUZ DE IGNACIO DE LA LLAVE,MUJER,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,AMBULATORIO,...,NaN,NO ESPECIFICADO,México,99,NO APLICA,41-50,NaN,NaN,Positivo SARS-CoV-2,NaN
3,2020-06-24,190217,USMER,SSA,SONORA,MUJER,SONORA,SONORA,NaN,HOSPITALIZADO,...,NaN,NO ESPECIFICADO,México,99,NO,41-50,NaN,1.0,Positivo SARS-CoV-2,NaN
4,2020-06-24,169761,USMER,SSA,YUCATÁN,HOMBRE,YUCATÁN,YUCATÁN,NaN,AMBULATORIO,...,NaN,NO ESPECIFICADO,México,99,NO APLICA,31-40,NaN,NaN,Positivo SARS-CoV-2,NaN


In [3]:
clean_covid_data = covid_cases_data[covid_cases_data["Negativo"]!="No positivo SARS-CoV-2"]
clean_covid_data = clean_covid_data[["ID_REGISTRO","SEXO","MUNICIPIO RESIDENCIA","FECHA INGRESO", "FECHA DEFUNCION","INTUBADO","EDAD","DIABETES","EPOC","ASMA","INMUNOSUPRESION","HIPERTENSION","CARDIOVASCULAR","OBESIDAD","RENAL CRONICA","TABAQUISMO"]]
clean_covid_data = clean_covid_data.fillna("NO")
display(clean_covid_data)

,ID_REGISTRO,SEXO,MUNICIPIO RESIDENCIA,FECHA INGRESO,FECHA DEFUNCION,INTUBADO,EDAD,DIABETES,EPOC,ASMA,INMUNOSUPRESION,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL CRONICA,TABAQUISMO
0,076847,MUJER,NO,2020-06-01,NO,NO APLICA,36,NO,NO,NO,NO,NO,NO,NO,NO,NO
1,0f2b37,HOMBRE,NO,2020-06-02,NO,NO APLICA,32,NO,NO,NO,NO,NO,NO,NO,NO,NO
2,1aa59e,MUJER,NO,2020-06-15,NO,NO APLICA,43,NO,NO,NO,NO,NO,NO,NO,NO,NO
3,190217,MUJER,NO,2020-06-13,NO,NO,42,NO,NO,NO,NO,SI,NO,NO,NO,NO
4,169761,HOMBRE,NO,2020-06-16,NO,NO APLICA,33,NO,NO,NO,NO,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490621,0db5d1,MUJER,NO,2020-05-28,NO,NO APLICA,60,SI,NO,NO,NO,NO,NO,NO,NO,NO
490622,1dc64a,HOMBRE,NO,2020-05-27,NO,NO APLICA,32,NO,NO,NO,NO,NO,NO,SI,NO,NO
490623,130185,MUJER,Iztapalapa,2020-05-12,NO,NO APLICA,52,NO,NO,NO,NO,NO,NO,NO,NO,NO
490624,186960,MUJER,Iztacalco,2020-04-15,NO,NO APLICA,47,NO,NO,NO,NO,NO,NO,SI,NO,NO


In [4]:
bins = [-1, 15, 20, 30, 40, 50, 60, 70, 80, 90, 999]
group_labels=['0-15','16-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91+']
data=[]
generos = ["TODOS","MUJER","HOMBRE"]
condiciones = ["TODOS","SIN CONDICION","DIABETES","EPOC","ASMA","INMUNOSUPRESION","HIPERTENSION","CARDIOVASCULAR","OBESIDAD","RENAL CRONICA","TABAQUISMO"]
datos = ["TODOS","INTUBADOS","DIFUNTOS"]
for condicion in condiciones:
    for dato in datos:
        for genero in generos:
            resumen = clean_covid_data.copy()
            if genero != "TODOS":
                resumen = resumen[resumen["SEXO"]==genero]
            if condicion != "TODOS":
                if condicion == "SIN CONDICION":
                    resumen = resumen[(resumen['DIABETES']=="NO") & (resumen['TABAQUISMO']=="NO") & (resumen['EPOC']=="NO") & (resumen['ASMA']=="NO") & (resumen['INMUNOSUPRESION']=="NO") & (resumen['HIPERTENSION']=="NO") & (resumen['CARDIOVASCULAR']=="NO") & (resumen['RENAL CRONICA']=="NO") & (resumen['OBESIDAD']=="NO")]
                else:
                    resumen = resumen[resumen[condicion]=="SI"]
            if dato == "INTUBADOS":
                resumen = resumen[resumen["INTUBADO"]=="SI"]
            if dato == "DIFUNTOS":
                resumen = resumen[resumen["FECHA DEFUNCION"]!="NO"]
            resumen["RANGO DE EDAD"] = pd.cut(resumen["EDAD"], bins, labels=group_labels)
            resumen = resumen.groupby(['RANGO DE EDAD']).count().rename(columns={'ID_REGISTRO':f'{genero} - {condicion} - {dato}'}).drop(['SEXO','FECHA DEFUNCION','MUNICIPIO RESIDENCIA','FECHA INGRESO','INTUBADO','EDAD','DIABETES','EPOC','ASMA','INMUNOSUPRESION','HIPERTENSION','CARDIOVASCULAR','OBESIDAD','RENAL CRONICA','TABAQUISMO'],axis=1).reset_index()
            resumen = resumen.transpose()
            resumen = resumen.rename(columns=resumen.iloc[0])
            resumen = resumen.drop(["RANGO DE EDAD"])
            data.append(resumen)
analisis_enfermedades = pd.concat(data)
analisis_enfermedades["TOTAL"] = analisis_enfermedades["0-15"]+analisis_enfermedades["16-20"]+analisis_enfermedades["21-30"]+analisis_enfermedades["31-40"]+analisis_enfermedades["41-50"]+analisis_enfermedades["51-60"]+analisis_enfermedades["61-70"]+analisis_enfermedades["71-80"]+analisis_enfermedades["81-90"]+analisis_enfermedades["91+"]
analisis_enfermedades = analisis_enfermedades.rename_axis("Grupo").reset_index()
separado = analisis_enfermedades["Grupo"].str.split(" - ", n = 2, expand = True) 
analisis_enfermedades["CONDICION"]= separado[1]
analisis_enfermedades["DATO"]= separado[2]
analisis_enfermedades["GENERO"]= separado[0]
analisis_enfermedades.drop(columns =["Grupo"], inplace = True)
analisis_enfermedades = analisis_enfermedades[['CONDICION','DATO','GENERO','0-15','16-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91+','TOTAL']]
display(analisis_enfermedades.head(15))

,CONDICION,DATO,GENERO,0-15,16-20,21-30,31-40,41-50,51-60,61-70,71-80,81-90,91+,TOTAL
0,TODOS,TODOS,TODOS,5778,4902,40326,58602,57744,45469,26952,13804,5020,725,259322
1,TODOS,TODOS,MUJER,2736,2574,20114,27686,26604,19825,11352,5803,2285,366,119345
2,TODOS,TODOS,HOMBRE,3042,2328,20212,30916,31140,25644,15600,8001,2735,359,139977
3,TODOS,INTUBADOS,TODOS,127,24,180,502,1164,1633,1508,874,270,27,6309
4,TODOS,INTUBADOS,MUJER,59,11,62,143,302,490,511,306,109,13,2006
5,TODOS,INTUBADOS,HOMBRE,68,13,118,359,862,1143,997,568,161,14,4303
6,TODOS,DIFUNTOS,TODOS,91,31,397,1476,3951,6537,6781,4783,1938,233,26218
7,TODOS,DIFUNTOS,MUJER,39,18,153,433,1143,2040,2410,1800,815,93,8944
8,TODOS,DIFUNTOS,HOMBRE,52,13,244,1043,2808,4497,4371,2983,1123,140,17274
9,SIN CONDICION,TODOS,TODOS,5010,3952,29713,38884,31371,19451,8677,3629,1397,248,142332


In [5]:
pd.DataFrame.to_csv(analisis_enfermedades,'../assets/data/enfermedades.csv',encoding='latin1')
analisis_enfermedades.to_json('../assets/data/enfermedades.json',orient='records')

In [6]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

#DATABASE
covid = myclient.Covid_DB

#COLLECTION
diseases=covid.diseases

data_dict = analisis_enfermedades.to_dict("records")

for record in data_dict:
    diseases.insert_one(record)
    
data_dict

[{'CONDICION': 'TODOS',
  'DATO': 'TODOS',
  'GENERO': 'TODOS',
  '0-15': 5778,
  '16-20': 4902,
  '21-30': 40326,
  '31-40': 58602,
  '41-50': 57744,
  '51-60': 45469,
  '61-70': 26952,
  '71-80': 13804,
  '81-90': 5020,
  '91+': 725,
  'TOTAL': 259322,
  '_id': ObjectId('5efa5a63fbab191678a66b3e')},
 {'CONDICION': 'TODOS',
  'DATO': 'TODOS',
  'GENERO': 'MUJER',
  '0-15': 2736,
  '16-20': 2574,
  '21-30': 20114,
  '31-40': 27686,
  '41-50': 26604,
  '51-60': 19825,
  '61-70': 11352,
  '71-80': 5803,
  '81-90': 2285,
  '91+': 366,
  'TOTAL': 119345,
  '_id': ObjectId('5efa5a64fbab191678a66b3f')},
 {'CONDICION': 'TODOS',
  'DATO': 'TODOS',
  'GENERO': 'HOMBRE',
  '0-15': 3042,
  '16-20': 2328,
  '21-30': 20212,
  '31-40': 30916,
  '41-50': 31140,
  '51-60': 25644,
  '61-70': 15600,
  '71-80': 8001,
  '81-90': 2735,
  '91+': 359,
  'TOTAL': 139977,
  '_id': ObjectId('5efa5a64fbab191678a66b40')},
 {'CONDICION': 'TODOS',
  'DATO': 'INTUBADOS',
  'GENERO': 'TODOS',
  '0-15': 127,
  '16-20'